### 팔레트 추출 및 상품 매칭 통합 코드
    
    최초 작성일 : 20/04/04
    작성자 : 김은주
    
    작성내용 : 팔레트 추출 및 상품 매칭 통합 코드 (CAIProjectModule.py)
               
               personal_color_palette 모듈화 및 테스트
               -> ColorPalette클래스 (palette_bright / palette_harmony / matchedPrdt / to_rgb / show_color)
               -> DatabaseConnetion클래스(connectDB / select_prdt_color / getPrdtimage)
               
               Main_product_color_extraction 모듈화 및 테스트
               -> ColorClustering클래스 (greytone_filter / color_generator / colorClustering)
               -> DatabaseConnection클래스 (getUserPurchase)
               
               최종 리턴 필요 값 추출 코드 정리
               -> 팔레트 별 웹용 RGB 값
               -> 추천항목 별 웹용 이미지링크

In [1]:
import CAIProjectModule

CP = CAIProjectModule.ColorPalette()
CD = CAIProjectModule.DatabaseConnetion()
CC = CAIProjectModule.ColorClustering()

In [5]:
### DB 연결
# database = {"host":"49.142.181.65", "user":"cai", "passwd":"1234", "db":"final"}
database = {"host":"localhost", "user":"root", "passwd":"1111", "db":"musinsa_test"}
cursor = CD.connectDB(database)

### PRDT_IMAGE 테이블에 H, S, V, 상품코드 조회
HSV = CD.select_prdt_color(cursor)

########################################### 컬러 팔레트 추출 #########################################
# 얼굴 측색결과 hsv 받아와야함
hsv = [27, 45, 80]
############################

### 얼굴 측색값 기준 팔레트 추출 : 각 6개 색 추출
hsv_palette_bright = CP.palette_bright(hsv)
hsv_palette_harmony = CP.palette_harmony(hsv)

### 팔레트 색 확인
# CP.show_colors(hsv_palette_bright)
# CP.show_colors(hsv_palette_harmony)

### 웹용 팔레트 RGB 변환 : 각 6개 색
rgb_palette_bright = CP.to_rgb(hsv_palette_bright)
rgb_palette_harmony = CP.to_rgb(hsv_palette_harmony)


########################################### 주요 구매색 추출 #########################################
# 웹에서 받아와야함
user_nick = "JAY"
############################

### 유저 닉네임에 따른 구매목록 조회
purchase_prdt = CD.getUserPurchase(cursor, user_nick)
### 회색조 필터링 (흰색/검은색 상품 필터링)
purchase_hsv = CC.greytoneFilter(purchase_prdt)
### Clustering 정확도를 위한 색값 제너레이팅
purchase_HSV = CC.colorGenerator(purchase_hsv)
### Clustering 중앙값(주구매색) 리턴
centroid = CC.colorClustering(purchase_HSV)


########################################### 상품매칭 & 이미지  #########################################

### 팔레트 컬러 상품 매칭 : 각 상품코드 6개
prdtCode_bright = CP.matchedPrdt(hsv_palette_bright, HSV)
prdtCode_harmony = CP.matchedPrdt(hsv_palette_harmony, HSV)
prdtCode_purchase = CP.matchedPrdt(centroid, HSV)

### 상품 이미지링크 갖고오기 : 웹페이지 노출용 이미지링크 각 6개
imgLink_bright = CD.getPrdtimage(cursor, prdtCode_bright)
imgLink_harmony = CD.getPrdtimage(cursor, prdtCode_harmony)
imgLink_purchase = CD.getPrdtimage(cursor, prdtCode_purchase)

In [7]:
# 웹용 컬러팔레트 RGB값
print("bright : ", rgb_palette_bright)
print("harmony : ", rgb_palette_harmony)

bright :  [[84.0, 127.0, 153.0], [217.0, 215.0, 184.0], [178.0, 116.0, 118.0], [129.0, 153.0, 69.0], [108.0, 167.0, 217.0], [94.0, 69.0, 153.0]]
harmony :  [[124.0, 84.0, 153.0], [184.0, 217.0, 189.0], [169.0, 178.0, 116.0], [69.0, 153.0, 110.0], [180.0, 108.0, 217.0], [153.0, 69.0, 111.0]]


In [9]:
# 웹용 이미지 링크
print("bright : ", imgLink_bright)
print("\nharmony : ", imgLink_harmony)
print("\npurchase : ", imgLink_purchase)

bright :  ['https://image.msscdn.net/images/goods_img/20181104/897632/897632_1_500.jpg', 'https://image.msscdn.net/images/goods_img/20190123/938783/938783_1_500.jpg', 'https://image.msscdn.net/images/goods_img/20191021/1194852/1194852_1_500.jpg', 'https://image.msscdn.net/images/goods_img/20200317/1355555/1355555_1_500.jpg', 'https://image.msscdn.net/images/goods_img/20200214/1307039/1307039_1_500.jpg', 'https://image.msscdn.net/images/goods_img/20200324/1365816/1365816_2_500.jpg']

harmony :  ['https://image.msscdn.net/images/goods_img/20180810/826451/826451_8_500.jpg', 'https://image.msscdn.net/images/goods_img/20180810/826451/826451_8_500.jpg', 'https://image.msscdn.net/images/goods_img/20180810/826451/826451_8_500.jpg', 'https://image.msscdn.net/images/goods_img/20180810/826451/826451_8_500.jpg', 'https://image.msscdn.net/images/goods_img/20180810/826451/826451_8_500.jpg', 'https://image.msscdn.net/images/goods_img/20180810/826451/826451_8_500.jpg']

purchase :  ['https://image.mss